<a href="https://colab.research.google.com/github/SerSanC/Flights-and-Airports-Data-classification-with-Spark/blob/main/Flights_classification_with_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

from pyspark.ml import Pipeline
#from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, MinMaxScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

spark = SparkSession.builder.master("local[*]").getOrCreate()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 42 kB/s 
     |████████████████████████████████| 199 kB 54.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=c31da3fcc66242b7e616c8699a493ebfb5e57c68176c994f5a90f5e6196852d0
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [2]:
from google.colab import files

uploaded = files.upload()

Saving flights.csv to flights.csv


## Load data

In [3]:
flights_df = spark.read.csv('flights.csv', inferSchema = True, header = True)
flights_df.show(10)

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          11433|        13303|      -3|       1|
|        19|        5|     DL|          14869|        12478|       0|      -8|
|        19|        5|     DL|          14057|        14869|      -4|     -15|
|        19|        5|     DL|          15016|        11433|      28|      24|
|        19|        5|     DL|          11193|        12892|      -6|     -11|
|        19|        5|     DL|          10397|        15016|      -1|     -19|
|        19|        5|     DL|          15016|        10397|       0|      -1|
|        19|        5|     DL|          10397|        14869|      15|      24|
|        19|        5|     DL|          10397|        10423|      33|      34|
|        19|        5|     DL|          11278|      

In [4]:
data = flights_df.select("DayofMonth", "DayOfWeek", "Carrier", "OriginAirportID", "DestAirportID", "DepDelay", ((col("ArrDelay") > 15).cast("Int").alias("label")))
data.show(10)

+----------+---------+-------+---------------+-------------+--------+-----+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|label|
+----------+---------+-------+---------------+-------------+--------+-----+
|        19|        5|     DL|          11433|        13303|      -3|    0|
|        19|        5|     DL|          14869|        12478|       0|    0|
|        19|        5|     DL|          14057|        14869|      -4|    0|
|        19|        5|     DL|          15016|        11433|      28|    1|
|        19|        5|     DL|          11193|        12892|      -6|    0|
|        19|        5|     DL|          10397|        15016|      -1|    0|
|        19|        5|     DL|          15016|        10397|       0|    0|
|        19|        5|     DL|          10397|        14869|      15|    1|
|        19|        5|     DL|          10397|        10423|      33|    1|
|        19|        5|     DL|          11278|        10397|     323|    1|
+----------+

### Split the Data
I will use 70% of the data for training, and reserve 30% for testing. In the testing data, the label column is renamed to trueLabel so I can use it later to compare predicted labels with known actual values.

In [8]:
splits = data.randomSplit([0.7, 0.3])
print(splits[0])
train = splits[0]
test = splits[1].withColumnRenamed("label", "trueLabel")
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, " Testing Rows:", test_rows)

DataFrame[DayofMonth: int, DayOfWeek: int, Carrier: string, OriginAirportID: int, DestAirportID: int, DepDelay: int, label: int]
Training Rows: 1892584  Testing Rows: 809634


In [10]:
train.show(10)

+----------+---------+-------+---------------+-------------+--------+-----+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|label|
+----------+---------+-------+---------------+-------------+--------+-----+
|         1|        1|     9E|          10397|        10693|      -2|    0|
|         1|        1|     9E|          10397|        12191|      -3|    0|
|         1|        1|     9E|          10397|        12191|      -2|    0|
|         1|        1|     9E|          10397|        12264|      -5|    0|
|         1|        1|     9E|          10397|        12264|      18|    0|
|         1|        1|     9E|          10397|        12264|      94|    1|
|         1|        1|     9E|          10397|        13851|       2|    0|
|         1|        1|     9E|          10423|        11433|      -5|    0|
|         1|        1|     9E|          10423|        11433|      -5|    0|
|         1|        1|     9E|          10423|        13244|      -6|    0|
+----------+

In [11]:
test.show(10)

+----------+---------+-------+---------------+-------------+--------+---------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|trueLabel|
+----------+---------+-------+---------------+-------------+--------+---------+
|         1|        1|     9E|          10397|        12264|      -3|        0|
|         1|        1|     9E|          10397|        12264|      23|        0|
|         1|        1|     9E|          10397|        13851|      -2|        0|
|         1|        1|     9E|          10423|        11433|      -3|        0|
|         1|        1|     9E|          10423|        11433|      14|        1|
|         1|        1|     9E|          10423|        13487|     -10|        0|
|         1|        1|     9E|          10423|        13487|      -7|        0|
|         1|        1|     9E|          10423|        13487|      -6|        0|
|         1|        1|     9E|          10423|        13487|      -4|        0|
|         1|        1|     9E|          